# 20.基于模型的策略优化 (model-based policy optimization，MBPO）
- **Dyna-Q** 算法中的模型只存储之前遇到的数据，只适用于 **表格型环境**；
- 在 **连续型** 状态和动作的环境中，若继续利用 **Dyna** 的思想，可以像 **PETS** 算法一样学习一个用 **神经网络环境模型**，然后在任意状态和动作下用 **环境模型** 来生成一些虚拟数据来帮助进行策略的学习；
- 如此，对 **真实环境** 中样本的需求量会减少，通常会比 **无模型** 的强化学习方法具有更高的采样效率 —— **MBPO** 算法。

## 20.1 MBPO 原理
> **加州大学伯克利分校 (UC Berkeley)** 的研究员在 2019 年的 NeurIPS 会议中提出 **MBPO** 算法 [When to Trust Your Model: Model-Based Policy Optimization](https://arxiv.org/abs/1906.08253) 

> 研究证明，在多个 **MuJoCo** 连续控制任务上，**MBPO** 的性能相比 **基于模型** 的方法更优，甚至在样本效率上超过了一些 **无模型算法**（如 **SAC**）
> **MBPO** 被认为是 **基于模型的强化学习领域中的里程碑式算法**，成为深度强化学习中最重要的**基于模型**的强化学习算法之一，不少之后的工作都是在此基础上进行的

> **MBPO** 算法基于 **两个关键** 的观察：
- 随着环境模型的 **推演步数** 变长，模型累积的 **复合误差** 会快速增加，使得环境模型得出的结果变得很 **不可靠**
- **推演步数** 必须要 **权衡**  *推演步数长，复合误差大* 的 **负面作用** 与 *推演步数长，训练策略更优* 的 **正面作用**
> 在这两个观察的基础之上提出了一种 **分支推演（branched rollout）**  的思想：*只使用环境模型 **从之前访问过的真实状态开始** 进行 **较短步数** 的推演，**而非**从初始状态开始进行**完整的推演**，**避免**长时间依赖预测，**减弱**模型误差累积的影响：*
![示意图](Illustrations/分支推演示意图.png)
- **分支推演的长度$k$** 是 **平衡** 样本效率和策略性能的 **重要超参数**

### *真实环境下策略性能提升的单调性保障*
> 基于模型的方法往往是在 **环境模型** 中提升策略的性能，但这并不能保证在 **真实环境** 中策略性能也有所提升，对此需要保证：

$$\eta[\pi]\geq\hat{\eta}[\pi]-\left[\frac{2\gamma r_{\max}(\epsilon_m+2\epsilon_\pi)}{(1-\gamma)^2}+\frac{4r_{\max}\epsilon_\pi}{(1-\gamma)}\right]$$
$$C=\left[\frac{2\gamma r_{\max}(\epsilon_m+2\epsilon_\pi)}{(1-\gamma)^2}+\frac{4r_{\max}\epsilon_\pi}{(1-\gamma)}\right]$$

- $\eta[\pi]$表示策略在 **真实环境** 中的 **期望回报**
- $\hat{\eta}[\pi]$表示策略在 **模型环境** 中的 **期望回报**
- $\epsilon_m$: **模型误差**，指模型在预测环境动态时的误差度量，代表着 **模型预测** 与 **真实环境状态转移**之间的差异（单步误差）
- $\epsilon_\pi$: **策略误差**，指策略执行时的误差度量，刻画了在 **同一当前策略$\pi$** 下，**模型预测的状态轨迹** 与 **真实轨迹** 之间的偏移（长期误差）

> 误差项的构造：
- $\frac{2\gamma r_{\max}(\epsilon_m + 2\epsilon_\pi)}{(1-\gamma)^2}$ 捕捉 **“模型不准 + 策略偏移”** 引起的长期累积误差
- $\frac{4r_{\max} \epsilon_\pi}{(1-\gamma)}$ 捕捉 **“即使模型准，但分布不同”** 造成的性能差
- 具体的推导过程见原论文附录 **Appendix A**

> 这意味着如果 **模型环境** 策略性能的提升可以超过 **performance bound（性能界/性能下界）$C$** ，理论上就可以在 **真实环境** 中取得策略性能的提升

### *模型推演长度*
> **MBPO** 对 **模型误差 $\epsilon_m$** 进行了优化：使用 **策略相关模型误差$\epsilon_{m'}$** ，关注 **当前策略$\pi$** 在 **有限步 rollout（推演片段）** 下可能访问的分布中与真实分布的最大差异，而不是将在 **环境模型** 上训练策略时所触及到的 **所有分布** 都考虑进来。
> 此时，两种误差都被局限在 **当前策略的有限步访问分布** 中，将两者结合起来，可更准确地估计策略在模型里的性能差异，从而指导安全的 **rollout步长$H$** 和 **策略优化**：
![策略偏移与模型误差](Illustrations/策略偏移与模型误差.png)

> 对 $\epsilon_m'$ 在 $\epsilon_\pi=0$ 附近做一阶展开：
$$\epsilon_m^{\prime}(\epsilon_\pi)\approx\epsilon_m^{\prime}|_{\epsilon_\pi=0}+\epsilon_\pi\frac{\mathrm{d}\epsilon_m^{\prime}}{\mathrm{d}\epsilon_\pi}$$
- 并不是数学上严格意义的泰勒一阶展开，而是一种 **改进的近似**，只是为了 **捕捉策略偏移对模型误差的增量影响**（直观作用类似一阶展开）
- 没有闭式表达式能把 $\epsilon_m'$ 精确写成 $\epsilon_\pi$ 的函数


> 当策略偏移为零时，**环境误差** 退化，考虑 **全局的分布**：
$$\epsilon_m^{\prime}(\epsilon_\pi)\approx\epsilon_m+\epsilon_\pi\frac{\mathrm{d}\epsilon_m^{\prime}}{\mathrm{d}\epsilon_\pi}.$$

> 再结合上 $k$ 步 **分支推演**，得到一个 **新的策略期望回报界**：
$$\eta[\pi]\geq\eta^\mathrm{branch}[\pi]-2r_\mathrm{max}\left[\frac{\gamma^{k+1}\epsilon_\pi}{(1-\gamma)^2}+\frac{\gamma^k\epsilon_\pi}{(1-\gamma)}+\frac{k}{1-\gamma}\epsilon_{m^{\prime}}\right]$$

- 具体的推导过程见原论文附录 **Appendix A**

> 此时，在策略误差一定时，当推演步数$H$变大，$\frac{\gamma^{k+1}\epsilon_\pi}{(1-\gamma)^2}+\frac{\gamma^k\epsilon_\pi}{(1-\gamma)}$减小
> 虽然$\frac{k}{1-\gamma}\epsilon_{m^{\prime}}$ 增大，但如果 **策略转移损害对模型准确度的影响$\frac{\mathrm{d}\epsilon_m^{\prime}}{\mathrm{d}\epsilon_\pi}$** 足够小（在主流的机器人运动环境 **Mojoco** 的典型场景中，其数量级非常小，大约都在$[10^{-4},10^{-2}]$）
> 那么就存在一个 **正推演步长$k$**，使 **策略提升** 有效。



> 需要注意的是，在高随机性的离散状态环境中，往往环境模型的拟合精度较低，以至于$\frac{\mathrm{d}\epsilon_m^{\prime}}{\mathrm{d}\epsilon_\pi}$较大，此时使用基于 **分支推演** 的方法 **效果有限**

## 20.2 MBPO 代码实践(Pendulum-v1)

1. **MBPO** 算法与 **Dyna-Q** 算法十分类似。但 **Dyna-Q** 采用的**无模型强化学习部分**是 **Q-learning**，而 **MBPO** 是基于 **SAC**
2. **MBPO** 算法关于 **环境模型的构建** 和 **PETS** 算法中一致，都使用 **模型集成** 的方式
3. **MBPO** 算法使用了 **模型生成的数据** 和 **真实环境的数据** 混合训练策略，提升了 **样本效率** 和 **稳定性**

导入相关库：

In [2]:
# 基本库
import numpy as np
import random

import itertools
from itertools import count
import collections
from collections import namedtuple

# 神经网络
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.distributions.normal import Normal
# Gymnasium 是一个用于开发和测试强化学习算法的工具库，为 OpenAI Gym 的更新版本（2021迁移开发）
import gymnasium as gym

####  SAC 算法部分

**

In [ ]:
class PolicyNetContinuous(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim, action_bound):
        super(PolicyNetContinuous, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc_mu = torch.nn.Linear(hidden_dim, action_dim)
        self.fc_std = torch.nn.Linear(hidden_dim, action_dim)
        self.action_bound = action_bound

    def forward(self, x):
        x = F.relu(self.fc1(x))
        mu = self.fc_mu(x)
        std = F.softplus(self.fc_std(x))
        
        dist = Normal(mu, std)  # 构建高斯分布
        normal_sample = dist.rsample()  # rsample()是重参数化采样
        log_prob = dist.log_prob(normal_sample)
        # 动作范围映射
        action = torch.tanh(normal_sample)  # a=tanh(z) 
        action = action * self.action_bound
        # 概率密度的变量替换
        log_prob = log_prob - torch.log(1 - torch.tanh(action).pow(2) + 1e-7)  # 1-tanh(z)^2是tanh的导数（sech²），加1e-7避免log(0)
        return action, log_prob


class QValueNetContinuous(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(QValueNetContinuous, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim + action_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc_out = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x, a):
        cat = torch.cat([x, a], dim=1)
        x = F.relu(self.fc1(cat))
        x = F.relu(self.fc2(x))
        return self.fc_out(x)